In [1]:
import os
import numpy as np
import h5py

def import_and_save_data(darkcount_directory, fluorescence_directory, experiment_title, output_directory):
    """
    Import darkcount and fluorescence data from H5 files and save as NPY files.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Import and save darkcount data
    darkcount_data, dc_exposure_times = import_darkcount_data(darkcount_directory)

    # Squeeze the darkcount data to remove the extra dimension
    darkcount_data = np.squeeze(darkcount_data)
    
    np.save(os.path.join(output_directory, 'darkcount_data.npy'), darkcount_data)
    np.save(os.path.join(output_directory, 'darkcount_exposure_times.npy'), dc_exposure_times)
    print("Darkcount data saved.")

    # Import and save fluorescence data
    fluorescence_data, fl_exposure_times = import_fluorescence_data(fluorescence_directory, experiment_title)
    
    for condition, data in fluorescence_data.items():
        data = np.squeeze(data)
        np.save(os.path.join(output_directory, f'fluorescence_data_{condition}.npy'), data)
        np.save(os.path.join(output_directory, f'fluorescence_exposure_times_{condition}.npy'), fl_exposure_times[condition])
    print("Fluorescence data saved.")

def import_darkcount_data(directory):
    """
    Import darkcount data from H5 files in the specified directory.
    """
    darkcount_files = [f for f in os.listdir(directory) if f.startswith('darkcount') and f.endswith('.h5')]
    darkcount_files.sort(key=lambda x: float(x.split('_')[-1][:-3]))

    darkcount_data = []
    exposure_times = []

    for file in darkcount_files:
        file_path = os.path.join(directory, file)
        with h5py.File(file_path, 'r') as h5f:
            darkcount = h5f['Cube']['Images'][()]
            exposure_time = h5f['Cube']['TimeExposure'][()].item()
            darkcount_data.append(darkcount)
            exposure_times.append(exposure_time)

    return np.array(darkcount_data), np.squeeze(np.array(exposure_times))

def import_fluorescence_data(directory, experiment_title):
    """
    Import fluorescence data from H5 files in the specified directory,
    separating data by imaging conditions.
    """
    laser_wavelengths = {'1': '670', '2': '760', '3': '808'}
    emission_filters = {'12': 'BP1150', '13': 'BP1200', '14': 'BP1250', '15': 'BP1300', '16': 'BP1350', '17': 'BP1575'}

    # Initialize a dictionary to store the image file lists
    image_files = {}

    # Get the list of image files for each parameter combination
    for laser_key, laser_value in laser_wavelengths.items():
        for filter_key, filter_value in emission_filters.items():
            key = f"{experiment_title}_{laser_value}_{filter_value}"
            image_files[key] = [f for f in os.listdir(directory) if f.startswith(f"{experiment_title}_{laser_key}_{filter_key}")]
            image_files[key].sort(key=lambda x: float(x.split('_')[-1][:-3]))

    # Initialize dictionaries to store data and exposure times
    fluorescence_data = {}
    exposure_times = {}

    # Read the image files and store the data for each parameter combination
    for key, files in image_files.items():
        image_data = []
        times = []
        for file in files:
            file_path = os.path.join(directory, file)
            with h5py.File(file_path, 'r') as h5f:
                image = h5f['Cube']['Images'][()]
                exposure_time = h5f['Cube']['TimeExposure'][()].item()
                image_data.append(image)
                times.append(exposure_time)
        fluorescence_data[key] = np.squeeze(np.array(image_data))
        exposure_times[key] = np.array(times)

    return fluorescence_data, exposure_times

# Usage
darkcount_directory = '/Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AP/AP240717_darkcountmeasure_highgain'
fluorescence_directory = '/Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed'
experiment_title = 'Water_immersed'
output_directory = f'data/{experiment_title}'  # This will create a 'data' folder in your current directory

import_and_save_data(darkcount_directory, fluorescence_directory, experiment_title, output_directory)

Darkcount data saved.
Fluorescence data saved.
